In [1]:
import pandas as pd

import sys
sys.path.append('../../lib')

In [2]:
from taller_model_selection.evaluate import load_train_dev_test

(X_train, y_train), (X_dev, y_dev), test = load_train_dev_test('.')

{'pct(train)': 0.7837289649483001, 'pct(dev)': 0.11952685477518159, 'pct(test)': 0.09674418027651828}


In [3]:
from taller_model_selection.transformers import FeatureProjection, PretrainedFastTextTransformer
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

features_pipe =  make_union(
    PretrainedFastTextTransformer('title_fasttext.bin', 'title'),
    make_pipeline(
        FeatureProjection(['rooms', 'bedrooms', 'bathrooms', 'surface_total', 'surface_covered']),
        SimpleImputer()
    ),
    make_pipeline(
        FeatureProjection(['l3']), 
        SimpleImputer(strategy='most_frequent'),
        OneHotEncoder(sparse=False, drop='first')
    ), 
    make_pipeline(
        FeatureProjection(['l4']), 
        SimpleImputer(strategy='constant'),
        OneHotEncoder(sparse=False, drop='first')
    ), 
)

lr_pipe = make_pipeline(
    features_pipe,
    LinearRegression()
)


In [5]:
rf_pipe = make_pipeline(
    features_pipe,
    RandomForestRegressor(n_estimators=20)
)

In [7]:
lr_pipe.fit(X_train, y_train)
rf_pipe.fit(X_train, y_train);

Pipeline(steps=[('featureunion',
                 FeatureUnion(transformer_list=[('pretrainedfasttexttransformer',
                                                 PretrainedFastTextTransformer(field='title',
                                                                               fname='title_fasttext.bin')),
                                                ('pipeline-1',
                                                 Pipeline(steps=[('featureprojection',
                                                                  FeatureProjection(fields=['rooms',
                                                                                            'bedrooms',
                                                                                            'bathrooms',
                                                                                            'surface_total',
                                                                                            'surface_covered'])),

In [9]:
from taller_model_selection.metrics import rmse

def eval_pipe(model_name, pipe):
    return dict(
        name=model_name,
        train=rmse(y_train, pipe.predict(X_train)),
        dev=rmse(y_dev, pipe.predict(X_dev))
    )

In [8]:
df = pd.DataFrame([eval_pipe('lr', lr_pipe), eval_pipe('rf', rf_pipe)])
df

,name,train,dev
0,lr,392472.789838,358645.591731
1,rf,127622.563339,271146.933040
